# Bernstein-Vazirani Algorithm

## Problem Definition:

Given a Boolean function f:{0,1}^n -> {0,1}, there is always a way to write this function as the scalar product
<s,x>, where s is an unknown bitstring. We assume just as in Grover's algorithm, that there is an oracle, which can compute this function. The Bernstein-Vazirani (BV) algorithm then finds the hidden bitstring s with only one application of the oracle.

## Create the circuit

### Define the circuit structure.

In [31]:
# Import standard packages
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

# Create circuit.
qr = QuantumRegister(size=2, name='q')
anc = QuantumRegister(size=1, name='ancilla')
cr = ClassicalRegister(size=2, name='c')
qc = QuantumCircuit(qr, anc, c, name="Bernstein-Vazirani")

### Initialize the qubits.

In [32]:
# Initialize the qubits.
qc.h(qr[0:2])
qc.x(anc[0])
qc.h(anc[0])
qc.draw()

┌───┐     
    q_0: ┤ H ├─────
         ├───┤     
    q_1: ┤ H ├─────
         ├───┤┌───┐
ancilla: ┤ X ├┤ H ├
         └───┘└───┘
    c: 2/══════════

### Apply the oracle and finish the circuit
We choose a XOR-Gate (2 CNOT-Gates) as our oracle.
Therefore we are looking for s = |11>

In [36]:
# Add 2 CNOT-Gates to our circuit.
qc.cx(qr[0], anc[0])
qc.cx(qr[1], anc[0])

# Apply the Hadamard-Gates to the qubits again.
qc.h(qr[0:2])

# Measure the qubits.
qc.measure(qr[0:2], cr[0:2])

In [37]:
qc.draw()

┌───┐          ┌───┐     ┌─┐   
    q_0: ┤ H ├───────■──┤ H ├─────┤M├───
         ├───┤       │  └───┘┌───┐└╥┘┌─┐
    q_1: ┤ H ├───────┼────■──┤ H ├─╫─┤M├
         ├───┤┌───┐┌─┴─┐┌─┴─┐└───┘ ║ └╥┘
ancilla: ┤ X ├┤ H ├┤ X ├┤ X ├──────╫──╫─
         └───┘└───┘└───┘└───┘      ║  ║ 
    c: 2/══════════════════════════╩══╩═
                                   0  1

## Simulate the circuit. 

In [39]:
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit import transpile
from qiskit.primitives import StatevectorSampler

# Create simulator.
simulator = AerSimulator()


# Transpile circuit for simulator.
trans_circ = transpile(qc, simulator)

# Execute circuit.
result = simulator.run(trans_circ).result()
#result = StatevectorSampler().run([qc]).result()

# Count results.
counts = result.get_counts(trans_circ)

#print("\n Totalcount for states:", result[0].data.c.get_counts())

print(counts)

{'11': 1024}
